# Getting Started with Julia in Colab/Jupyter
You can either run this notebook in Google Colab, or using Jupyter on your own machine.

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia DataFrames CSV Pipe"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2022-01-09 12:01:53 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package DataFrames...
Installing Julia package CSV...
Installing Julia package Pipe...
Installing IJulia kernel...


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system (if you ever ask for help or file an issue about Julia, you should always provide this information).

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Step 1. Import the necessary libraries

In [2]:
using DataFrames
using CSV
using Pipe

ENV["column"] = 20

20

# Step 2. Import the dataset from this [address](https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv).


In [3]:
URL = "https://raw.githubusercontent.com/justmarkham/DAT8/master/data/chipotle.tsv"
download(URL, "chipotle.tsv")

"chipotle.tsv"

# Step 3. Assign it to a variable called chipo.


In [4]:
chipo = CSV.read("chipotle.tsv", DataFrame, delim = "\t")

,order_id,quantity,item_name,choice_description
,Int64,Int64,String,String
1,1,1,Chips and Fresh Tomato Salsa,NULL
2,1,1,Izze,[Clementine]
3,1,1,Nantucket Nectar,[Apple]
4,1,1,Chips and Tomatillo-Green Chili Salsa,NULL
5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]"
6,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sour Cream, Guacamole, Lettuce]]"
7,3,1,Side of Chips,NULL
8,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables, Black Beans, Pinto Beans, Cheese, Sour Cream, Guacamole, Lettuce]]"
9,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Cheese, Sour Cream, Lettuce]]"


# Step 4. How many products cost more than $10.00?


In [5]:
# Clean the $ before the price
transform!(chipo, :item_price => ByRow(x -> parse(Float32, x[2:end])) => :item_price)

,order_id,quantity,item_name,choice_description
,Int64,Int64,String,String
1,1,1,Chips and Fresh Tomato Salsa,NULL
2,1,1,Izze,[Clementine]
3,1,1,Nantucket Nectar,[Apple]
4,1,1,Chips and Tomatillo-Green Chili Salsa,NULL
5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]"
6,3,1,Chicken Bowl,"[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sour Cream, Guacamole, Lettuce]]"
7,3,1,Side of Chips,NULL
8,4,1,Steak Burrito,"[Tomatillo Red Chili Salsa, [Fajita Vegetables, Black Beans, Pinto Beans, Cheese, Sour Cream, Guacamole, Lettuce]]"
9,4,1,Steak Soft Tacos,"[Tomatillo Green Chili Salsa, [Pinto Beans, Cheese, Sour Cream, Lettuce]]"


In [6]:
greater_than10 = filter([:quantity, :item_price] => (x, y) -> x == 1 && y > 10, chipo)
unique(greater_than10[!,:item_name])

25-element Vector{String}:
 "Chicken Bowl"
 "Steak Burrito"
 "Chicken Burrito"
 "Barbacoa Bowl"
 "Veggie Burrito"
 "Veggie Bowl"
 "Chicken Soft Tacos"
 "Steak Bowl"
 "Carnitas Burrito"
 "Carnitas Bowl"
 "Barbacoa Burrito"
 "Barbacoa Crispy Tacos"
 "Veggie Salad Bowl"
 "Chicken Salad"
 "Chicken Crispy Tacos"
 "Steak Salad Bowl"
 "Veggie Soft Tacos"
 "Barbacoa Soft Tacos"
 "Carnitas Crispy Tacos"
 "Carnitas Salad Bowl"
 "Chicken Salad Bowl"
 "Barbacoa Salad Bowl"
 "Steak Soft Tacos"
 "Carnitas Soft Tacos"
 "Steak Crispy Tacos"

# Step 5. What is the price of each item?
print a data frame with only two columns item_name and item_price

In [15]:
items = chipo[chipo.quantity .== 1, [:item_name, :item_price]]
unique(items,1)

,item_name,item_price
,String,Float32
1,Chips and Fresh Tomato Salsa,2.39
2,Izze,3.39
3,Nantucket Nectar,3.39
4,Chips and Tomatillo-Green Chili Salsa,2.39
5,Chicken Bowl,10.98
6,Side of Chips,1.69
7,Steak Burrito,11.75
8,Steak Soft Tacos,9.25
9,Chips and Guacamole,4.45


# Step 6. Sort by the name of the item

In [17]:
sort(chipo,:item_name)

,order_id,quantity,item_name,choice_description,item_price
,Int64,Int64,String,String,Float32
1,129,1,6 Pack Soft Drink,[Sprite],6.49
2,148,1,6 Pack Soft Drink,[Diet Coke],6.49
3,154,1,6 Pack Soft Drink,[Coke],6.49
4,168,1,6 Pack Soft Drink,[Diet Coke],6.49
5,182,1,6 Pack Soft Drink,[Diet Coke],6.49
6,230,1,6 Pack Soft Drink,[Diet Coke],6.49
7,264,1,6 Pack Soft Drink,[Diet Coke],6.49
8,298,1,6 Pack Soft Drink,[Nestea],6.49
9,306,1,6 Pack Soft Drink,[Coke],6.49


# Step 7. What was the quantity of the most expensive item ordered?


In [26]:
sort(chipo, :item_price, rev = true)[1,[:item_name,:quantity]]

,item_name,quantity
,String,Int64
1,Chips and Fresh Tomato Salsa,15


# Step 8. How many times was a Veggie Salad Bowl ordered?


In [32]:
salad_orders = chipo[chipo.item_name.=="Veggie Salad Bowl", :]
nrow(salad_orders)

18

# Step 9. How many times did someone order more than one Canned Soda?

In [40]:
soda_orders = chipo[(chipo.item_name.=="Canned Soda").&(chipo.quantity.>1), :]
nrow(soda_orders)

20